# 28 - 03 - 2021
## First draft of our abstract

In our first meeting with the group, we discussed about the topic and created a first draft of our abstract. For this, everyone wrote a draft and we combined our results in the end.

# 30 - 03 - 2021
## Preliminary Analysis: exploring the classes of errors
I started to carry some preliminary analysis, in order to butter understand the problem that we need to tackle in our research question. The dataset that we need to use to solve our problem is provided by Silvio Peroni under public license at http://doi.org/10.5281/zenodo.4625300. This dataset contains a two-column CSV file, where the first column ("Valid_citing_DOI") contains the DOI of a citing entity retrieved in Crossref, while the second column ("Invalid_cited_DOI") contains the invalid DOI of a cited entity identified by looking at the field "reference" in the JSON document returned by querying the [Crossref API](https://www.crossref.org/education/retrieve-metadata/rest-api/) with the citing DOI. <br />
Among the column containing invalid cited DOIs I noticed some first general classes of errors that invalid the identifier:<br />
* **DOIs containing additional URLs:**
  1. 10.1016/j.aca.2006.07.086.http://dx.doi.org/10.1016/j.aca.2006.07.086 → 10.1016/j.aca.2006.07.086
  2. “10.1186/1735-2746-10-21,http://www.ijehse.com/content/10/1/21" → 10.1186/1735-2746-10-21
* **Extra characters at the end:**
  1. 10.1061/9780784480502.018] →  10.1061/9780784480502.018
  2. 10.1044/1092-4388(2012/11-0316)a →  10.1044/1092-4388(2012/11-0316)
* **Extra strings at the end:** 
  1. 10.1111/j.1099-0860.1997.tb00004.x/abstract> → 10.1111/j.1099-0860.1997.tb00004.x
  2. 10.4103/0975-�-7406.163460>accessed8 →  10.4103/0975-7406.163460
  3. 10.1007/s10706-019-01181-9(0123456789 → 10.1007/s10706-019-01181-9
* **DOIs with wrongly encoded HTML entities:**
  1. 10.1379/1466-1268(1997)002lt;0162:tmethi>2.3.co;2 → 10.1379/1466-1268(1997)002<0162:tmethi>2.3.co;2

Arcangelo noticed that some **DOIs contain queries to proxy servers or characters forbidden in URLs:**
  1. 10.2307/2491102?uid=37380728uid=20uid=40sid=4102564553863 → 10.2307/2491102
  2. 10.1016/j.envexpbot.2013.10.018#doilink → 10.1016/j.envexpbot.2013.10.018





# 03 - 04 - 2021
## First draft of Data Management Plan (DMP)
In a group meeting, we created a DMP on the platform Argos (https://argos.openaire.eu/) and divided the further work on it (the description of the two datasets) between us.
For the DMP, I focused on part **1: Data Summary** and **2: Reusable Data** of the dataset containing the source code used to clean a CSV list of invalid DOI names.

# 07 - 04 - 2021
## First bites of code: Counting the invalid DOI-to-DOI citations in the CSV

I started to carry some preliminary computations on our data, by counting how many pair of invalid DOI-to-DOI citations are in the CSV that we use as input data.

In [4]:
!wget https://zenodo.org/record/4625300/files/invalid_dois.csv

# if you are running this notebook locally on Windows 10 use:
# pip install wget
# !python -m wget https://zenodo.org/record/4625300/files/invalid_dois.csv

--2021-04-09 13:11:48--  https://zenodo.org/record/4625300/files/invalid_dois.csv
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64233224 (61M) [text/plain]
Saving to: ‘invalid_dois.csv’

invalid_dois.csv    100%[===================>]  61.26M  17.2MB/s    in 4.7s    

2021-04-09 13:11:54 (12.9 MB/s) - ‘invalid_dois.csv’ saved [64233224/64233224]



In [5]:
from csv import reader

f = reader(open('invalid_dois.csv'))
header = next(f)
input_list = [(doi1, doi2) for (doi1, doi2) in f]
print(input_list[:50])

[('10.14778/1920841.1920954', '10.5555/646836.708343'), ('10.5406/ethnomusicology.59.2.0202', '10.2307/20184517'), ('10.1161/01.cir.63.6.1391', '10.1161/circ.37.4.509'), ('10.1177/1179546820918903', '10.3748/wjg.v10.i5.707.'), ('10.1080/10410236.2020.1731937', '10.1070/10810730903528033'), ('10.1161/strokeaha.112.652065', '10.1161/str.24.7.8322400'), ('10.1177/1049732310393747', '10.1111/j.545-5300.2003.42208.x'), ('10.1155/2017/1491405', '10.3760/cma.j.issn.0366-6999.20131202'), ('10.1161/01.res.68.6.1549', '10.1161/res.35.2.159'), ('10.4018/978-1-5225-2650-6.ch006', '10.1002/per'), ('10.1145/2525314.2594229', '10.5555/1873601.1873616'), ('10.1007/s10619-020-07320-z', '10.3390/sym11070911www.mdpi.com/journal/symmetry'), ('10.1007/s11771-020-4410-2', '10.13745/j.esf.2016.02.011'), ('10.1161/01.cir.102.5.591', '10.1161/circ.85.3.1537115'), ('10.1007/s40617-018-00299-1', '10.1901/jaba.2012.45-657'), ('10.1074/jbc.m508416200', '10.1059/0003-4819-100-4-483'), ('10.1177/2054358119836124', '

In [6]:
print(len(input_list))

1223296


In [7]:
invalid_dois = list(zip(*input_list))[1]
print(invalid_dois[:50])

('10.5555/646836.708343', '10.2307/20184517', '10.1161/circ.37.4.509', '10.3748/wjg.v10.i5.707.', '10.1070/10810730903528033', '10.1161/str.24.7.8322400', '10.1111/j.545-5300.2003.42208.x', '10.3760/cma.j.issn.0366-6999.20131202', '10.1161/res.35.2.159', '10.1002/per', '10.5555/1873601.1873616', '10.3390/sym11070911www.mdpi.com/journal/symmetry', '10.13745/j.esf.2016.02.011', '10.1161/circ.85.3.1537115', '10.1901/jaba.2012.45-657', '10.1059/0003-4819-100-4-483', '10.1016/j.amepre.2015.07.017.', '10.1016/j.atmosenv.2008.0305', '10.1161/circ.66.1.7083497', '10.1080/1364253032000157166', '10.1161/str.23.11.1440702', '10.1186/s12933-015-0183-6.', '10.1037/0021–9010.93.3.602', '10.1177/0312896216656720.', '10.5555/2133036.2133123', '10.5555/2442626.2442634', '10.5555/1785594.1785635', '10.1161/res.59.2.2874900', '10.1002/smj', '10.1161/res.89.12.1216', '10.3390/brainsci7120164.', '10.5555/944919.944937', '10.1145/2838344.2856460', '10.1177/0890117116661982.', '10.1161/circ.39.1.48', '10.110

#09 - 04 - 2021
## Useful References

Here I will list the useful references that I found in order to carry a **literature review**:

1. Xu, S., Hao, L., An, X. et al. Types of DOI errors of cited references in Web of Science with a cleaning method. Scientometrics 120, 1427–1437 (2019). https://doi.org/10.1007/s11192-019-03162-4
2. Zhu, J., Hu, G. & Liu, W. DOI errors and possible solutions for Web of Science. Scientometrics 118, 709–718 (2019). https://doi.org/10.1007/s11192-018-2980-7 
3. Franceschini, F., Maisano, D., & Mastrogiacomo, L. (2015). Errors in DOI indexing by bibliometric databases. Scientometrics, 102(3), 2181–2186. https://doi.org/10.1007/s11192-014-1503-4.



